In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.1/8062.4 GB disk)


In [2]:
!pip uninstall -y wandb

Found existing installation: wandb 0.19.1
Uninstalling wandb-0.19.1:
  Successfully uninstalled wandb-0.19.1


In [ ]:
!yolo train model=yolov8m.pt data="/kaggle/input/pakistani-license-pate-dataset/data.yaml" epochs=50 imgsz=640 batch=8

In [3]:
from ultralytics import YOLO

# Load your YOLO model
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')  # Replace 'best.pt' with your model path

# Get class names
print("Model Classes:", model.names)


Model Classes: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'A', 11: 'B', 12: 'C', 13: 'D', 14: 'E', 15: 'F', 16: 'G', 17: 'H', 18: 'I', 19: 'J', 20: 'K', 21: 'L', 22: 'LicensePlate', 23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R', 29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'X', 35: 'Y', 36: 'Z'}


In [4]:
import shutil

# Path to the folder you want to zip
folder_path = '/kaggle/working/license_plate_output00'

# Output ZIP file path
zip_file_path = '/kaggle/working/license_plate_results11.zip'

# Create a ZIP file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Folder successfully zipped: {zip_file_path}")


Folder successfully zipped: /kaggle/working/license_plate_results11.zip


In [3]:
!pip install paddleocr
!pip install paddlepaddle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.7/544.7 kB 15.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.9/161.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 92.7 MB/s eta 0:00:00:00:010:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=48b884f298ab47f56ba2916b17f0382f6f714a97c58032066fd66c3a4546d99c
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninsta

## they all number and char 

In [17]:
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_6/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR accuracy
        gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(gray_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = "".join([word[1][0] for word in line])  # Extract detected text

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, plate_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        print(f"Detected Plate: {plate_text}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, f"result_{i}.png")
    cv2.imwrite(final_image_path, image)

print(f"All full images with detected plates are saved in '{output_folder}'.")


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:16<00:00, 231.51it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:17<00:00, 586.50it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:16<00:00, 133.10it/s]

[2025/02/24 09:38:52] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c


image 1/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0968.JPG: 576x640 1 LicensePlate, 35.7ms
image 2/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0969.JPG: 544x640 1 LicensePlate, 35.1ms
image 3/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0970.JPG: 576x640 1 LicensePlate, 35.4ms
image 4/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0971.JPG: 608x640 1 LicensePlate, 36.5ms
image 5/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0971x.jpg: 608x640 1 LicensePlate, 32.6ms
image 6/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0973.JPG: 576x640 1 LicensePlate, 20.6ms
image 7/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0973x.jpg: 512x640 1 LicensePlate, 16.7ms
image 8/104 /kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars/DSC_0975.JPG: 640x480 1 LicensePla

In [18]:
import shutil

# Path to the folder you want to zip
folder_path = '/kaggle/working/license_plate_output_16'

# Output ZIP file path
zip_file_path = '/kaggle/working/license_plate_results_16.zip'

# Create a ZIP file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Folder successfully zipped: {zip_file_path}")


Folder successfully zipped: /kaggle/working/license_plate_results_16.zip


save to csv

In [ ]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_7/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR accuracy
        gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(gray_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = "".join([word[1][0] for word in line])  # Extract detected text

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, plate_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, plate_text])

        print(f"Detected Plate: {plate_text}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")


## update

In [ ]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
import re
from paddleocr import PaddleOCR

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_9/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Function to extract valid registration number
def extract_registration_number(plate_text):
    # Find potential license plate formats
    matches = re.findall(r"[A-Z]{2,3}[-\s]?\d{3,4}", plate_text)
    if matches:
        return matches[0].replace("-", " ")  # Replace dash with space if needed
    return plate_text  # Return original text if no match (avoids "Unknown")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR accuracy
        gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(gray_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = " ".join([word[1][0] for word in line])  # Extract detected text

        # Extract only the main registration number
        final_plate = extract_registration_number(plate_text)

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, final_plate, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, final_plate])

        print(f"Detected Plate: {final_plate}")

    # Save the full image with cleaned license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")


In [ ]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR
import re  # For text filtering

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_10/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

def clean_plate_text(text):
    """
    Cleans the OCR output to extract only the registration number (e.g., 'LEA 1856').
    """
    # Define a regex pattern to match valid Pakistani license plates
    pattern = re.findall(r'([A-Z]{2,3})\s?(\d{3,4})', text)
    if pattern:
        return f"{pattern[0][0]} {pattern[0][1]}"  # Return formatted plate number
    return "UNKNOWN"

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR accuracy
        gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(gray_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = " ".join([word[1][0] for word in line])  # Extract detected text
        
        # Clean the extracted text
        cleaned_plate = clean_plate_text(plate_text)

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, cleaned_plate, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, cleaned_plate])

        print(f"Detected Plate: {cleaned_plate}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")


Update

In [8]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR
import re  # For text filtering

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/imagefortest/Pakistani License Number Plates Data/Cars'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_13/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

def preprocess_plate(plate_crop):
    """
    Apply preprocessing to the license plate image to improve OCR accuracy.
    """
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def clean_plate_text(text):
    """
    Cleans the OCR output to extract only the registration number (e.g., 'LEA 1856').
    """
    # Adjust the regex to better match the format of the license plate
    pattern = re.findall(r'([A-Z]{2,3})\s?(\d{3,4})', text)
    if pattern:
        return f"{pattern[0][0]} {pattern[0][1]}"  # Return formatted plate number
    return "UNKNOWN"

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Preprocess the plate for better OCR accuracy
        processed_plate = preprocess_plate(plate_crop)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(processed_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = " ".join([word[1][0] for word in line])  # Extract detected text
        
        # Print raw OCR output for debugging
        print(f"Raw OCR Output: {plate_text}")

        # Clean the extracted text
        cleaned_plate = clean_plate_text(plate_text)

        # If OCR result is "UNKNOWN", try alternative preprocessing
        if cleaned_plate == "UNKNOWN":
            _, alt_thresh = cv2.threshold(processed_plate, 127, 255, cv2.THRESH_BINARY_INV)
            ocr_result_alt = ocr.ocr(alt_thresh, cls=True)

            for line in ocr_result_alt:
                if line:
                    alt_text = " ".join([word[1][0] for word in line])
                    cleaned_plate = clean_plate_text(alt_text)

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, cleaned_plate, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, cleaned_plate])

        print(f"Detected Plate: {cleaned_plate}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")

[2025/02/25 10:15:37] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [25]:
import shutil

# Path to the folder you want to zip
folder_path = '/kaggle/working/license_plate_output_20'

# Output ZIP file path
zip_file_path = '/kaggle/working/license_for_video_plate_results_20.zip'

# Create a ZIP file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Folder successfully zipped: {zip_file_path}")


Folder successfully zipped: /kaggle/working/license_for_video_plate_results_20.zip


In [ ]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR
import re  # For text filtering

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/video-for-plate/Licence Plate Camera Illustration Video.mp4'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_15/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

def preprocess_plate(plate_crop):
    """
    Apply preprocessing to the license plate image to improve OCR accuracy.
    """
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def clean_plate_text(text):
    """
    Cleans the OCR output to extract only the registration number (e.g., 'LEA 1856').
    """
    # Adjust the regex to better match the format of the license plate
    pattern = re.findall(r'([A-Z]{2,3})\s?(\d{3,4})', text)
    if pattern:
        return f"{pattern[0][0]} {pattern[0][1]}"  # Return formatted plate number
    return "UNKNOWN"

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Preprocess the plate for better OCR accuracy
        processed_plate = preprocess_plate(plate_crop)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(processed_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = " ".join([word[1][0] for word in line])  # Extract detected text
        
        # Print raw OCR output for debugging
        print(f"Raw OCR Output: {plate_text}")

        # Clean the extracted text
        cleaned_plate = clean_plate_text(plate_text)

        # If OCR result is "UNKNOWN", try alternative preprocessing
        if cleaned_plate == "UNKNOWN":
            _, alt_thresh = cv2.threshold(processed_plate, 127, 255, cv2.THRESH_BINARY_INV)
            ocr_result_alt = ocr.ocr(alt_thresh, cls=True)

            for line in ocr_result_alt:
                if line:
                    alt_text = " ".join([word[1][0] for word in line])
                    cleaned_plate = clean_plate_text(alt_text)

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, cleaned_plate, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, cleaned_plate])

        print(f"Detected Plate: {cleaned_plate}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")

### update code for video

In [26]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR
import re
import numpy as np

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to video file
video_path = '/kaggle/input/2-video-for-test/v1.mp4'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_21/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# Initialize video capture
cap = cv2.VideoCapture(video_path)

# List to store results for CSV
plate_data = []
seen_plates = {}  # Store unique plates and their last seen frame

def preprocess_plate(plate_crop):
    """
    Apply preprocessing to the license plate image to improve OCR accuracy.
    """
    gray = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return thresh

def clean_plate_text(text):
    """
    Cleans the OCR output to extract only the registration number (e.g., 'LEA 1856').
    """
    pattern = re.findall(r'([A-Z]{2,3})\s?(\d{3,4})', text)
    if pattern:
        return f"{pattern[0][0]} {pattern[0][1]}"
    return "UNKNOWN"

def is_duplicate(plate_text, bbox, frame_count, threshold=50):
    """
    Checks if the detected plate is a duplicate based on IoU and time threshold.
    """
    global seen_plates
    for prev_plate, (prev_bbox, prev_frame) in seen_plates.items():
        if plate_text == prev_plate:
            # Compute IoU (Intersection over Union) to check if it's the same car
            x1, y1, x2, y2 = bbox
            px1, py1, px2, py2 = prev_bbox

            # Compute IoU
            xi1 = max(x1, px1)
            yi1 = max(y1, py1)
            xi2 = min(x2, px2)
            yi2 = min(y2, py2)

            inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            bbox1_area = (x2 - x1) * (y2 - y1)
            bbox2_area = (px2 - px1) * (py2 - py1)
            union_area = bbox1_area + bbox2_area - inter_area
            iou = inter_area / union_area

            # If IoU is high and seen recently, it's a duplicate
            if iou > 0.5 and (frame_count - prev_frame) < threshold:
                return True

    # Update the seen_plates dictionary
    seen_plates[plate_text] = (bbox, frame_count)
    return False

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    frame_count += 1
    results = model.predict(source=frame, save=False, classes=22)  # Detect only plates

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            plate_crop = frame[y1:y2, x1:x2]
            processed_plate = preprocess_plate(plate_crop)

            # Perform OCR
            ocr_result = ocr.ocr(processed_plate, cls=True)
            plate_text = ""
            for line in ocr_result:
                if line:
                    plate_text = " ".join([word[1][0] for word in line])

            cleaned_plate = clean_plate_text(plate_text)

            # Check for duplicates
            if cleaned_plate != "UNKNOWN" and not is_duplicate(cleaned_plate, (x1, y1, x2, y2), frame_count):
                # Draw bounding box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
                cv2.putText(frame, cleaned_plate, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

                # Save result
                plate_data.append([f"frame_{frame_count}.png", cleaned_plate])
                print(f"Detected Plate (Unique): {cleaned_plate}")

    # Save processed frame
    output_image_path = os.path.join(output_folder, f"frame_{frame_count}.png")
    cv2.imwrite(output_image_path, frame)

cap.release()
cv2.destroyAllWindows()

# Save results to CSV
df = pd.DataFrame(plate_data, columns=['Frame', 'Detected Plate'])
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"Results saved in {csv_file_path}")


[2025/02/25 11:20:41] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [29]:
import shutil

# Path to the folder you want to zip
folder_path = '/kaggle/working/license_plate_output_22'

# Output ZIP file path
zip_file_path = '/kaggle/working/license_for_video_plate_results_22.zip'

# Create a ZIP file
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Folder successfully zipped: {zip_file_path}")


Folder successfully zipped: /kaggle/working/license_for_video_plate_results_22.zip


In [28]:
import pandas as pd
from ultralytics import YOLO
import os
import cv2
from paddleocr import PaddleOCR

# Load the YOLO model (replace with your model path)
model = YOLO('/kaggle/input/pakistani_license_plate_detection/tensorflow2/default/1/runs/detect/train/weights/best.pt')

# Path to the folder containing images
image_folder = '/kaggle/input/2-video-for-test/v2.mp4'

# Create output directory for results
output_folder = '/kaggle/working/license_plate_output_22/'
os.makedirs(output_folder, exist_ok=True)

# Initialize PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang="en")  

# List to store results for CSV
plate_data = []

# Detect ONLY the license plate (class index 22)
results = model.predict(source=image_folder, save=False, classes=22)

print(f"License plate detection complete. Extracting text using PaddleOCR...")

# Process each image and extract license plate text
for i, result in enumerate(results):
    image = result.orig_img
    image_name = f"result_{i}.png"

    for j, box in enumerate(result.boxes):
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Extract the license plate region for OCR
        plate_crop = image[y1:y2, x1:x2]

        # Convert to grayscale for better OCR accuracy
        gray_plate = cv2.cvtColor(plate_crop, cv2.COLOR_BGR2GRAY)

        # Use PaddleOCR for text extraction
        ocr_result = ocr.ocr(gray_plate, cls=True)

        # Extract recognized text
        plate_text = ""
        for line in ocr_result:
            if line:
                plate_text = "".join([word[1][0] for word in line])  # Extract detected text

        # Draw the license plate bounding box on the original image
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 5)  # Green box

        # Draw the detected license plate number on the image
        font_scale = 1.0
        font_thickness = 2
        cv2.putText(image, plate_text, (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (0, 255, 0), font_thickness)

        # Save result in list
        plate_data.append([image_name, plate_text])

        print(f"Detected Plate: {plate_text}")

    # Save the full image with license plate text
    final_image_path = os.path.join(output_folder, image_name)
    cv2.imwrite(final_image_path, image)

# Convert list to DataFrame
df = pd.DataFrame(plate_data, columns=['Image Name', 'Detected Plate'])

# Save to CSV
csv_file_path = os.path.join(output_folder, "license_plate_data.csv")
df.to_csv(csv_file_path, index=False)

print(f"All full images with detected plates are saved in '{output_folder}'.")
print(f"License plate data saved in CSV file: {csv_file_path}")


[2025/02/25 11:25:13] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c